In [ ]:
import os
import sys

# Explicitly set the project root
project_root = '/Users/afshinpaydar/Desktop/test/saved_models'
# Add to Python path
sys.path.insert(0, project_root)
os.chdir('../')  # Moving up one directory to the root

In [ ]:
from main import SentimentAnalyzer
from config.model_config import Config 
from models.modelPersistence import ModelPersistence
from utils.modelEvaluator import ModelEvaluator

In [ ]:
modelPersistence = ModelPersistence()
analyzer = SentimentAnalyzer()

In [ ]:
# Create and setup analyzer
analyzer.process_data()

In [ ]:
# Load necessary imports at the top of your notebook cell
import os
import json
import tensorflow as tf
from transformers import DistilBertTokenizer
from models.sentiment_model import EnhancedDistilBertForSentiment
from config.model_config import Config

def load_saved_model(epoch, model_dir=None):
    """
    Load a previously saved model with weights and configuration.
    Args:
        epoch (int): The training epoch to load
        model_dir (str): Directory containing saved model files
    Returns:
        tuple: (loaded_model, training_history)
    """
    try:
        # Handle model directory path
        if model_dir is None:
            model_dir = os.path.join(os.getcwd(), 'saved_models')
            
        print(f"Loading model from directory: {model_dir}")
        
        # 1. Load model info and configuration
        model_info_path = os.path.join(model_dir, f"model_info_epoch{epoch}.json")
        if not os.path.exists(model_info_path):
            raise FileNotFoundError(f"Model info file not found at: {model_info_path}")
            
        with open(model_info_path, 'r') as f:
            model_info = json.load(f)
            print("Loaded model configuration and training history")
            
        # 2. Initialize tokenizer
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        print("Initialized tokenizer")
            
        # 3. Initialize and build model
        model = EnhancedDistilBertForSentiment()
        
        # Build model with dummy input
        dummy_input = {
            'input_ids': tf.zeros((1, Config.MAX_LENGTH), dtype=tf.int32),
            'attention_mask': tf.zeros((1, Config.MAX_LENGTH), dtype=tf.int32)
        }
        _ = model(dummy_input)
        print("Initialized model architecture")
            
        # 4. Load weights
        weights_path = os.path.join(model_dir, f"enhanced_distilbert_epoch{epoch}_weights")
        if not os.path.exists(weights_path + '.index'):
            raise FileNotFoundError(f"Model weights not found at: {weights_path}")
            
        model.load_weights(weights_path)
        print("Loaded model weights successfully")
            
        # 5. Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=Config.LEARNING_RATE),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        print("Model compiled successfully")
            
        print("\nModel loaded successfully!")
        print(f"Original training epochs: {model_info['epoch']}")
        print(f"Final validation accuracy: {model_info['history']['val_accuracy'][-1]:.4f}")
            
        return model, model_info['history']
        
    except Exception as e:
        print(f"\nError loading model: {str(e)}")
        print("Load location attempted: ", model_dir)
        raise

# Then in another cell, load the model:
epoch = 10
loaded_model, training_history = load_saved_model(epoch)

In [ ]:
# 3. loading the previously saved model:
epoch = 10  
loaded_model, training_history = load_saved_model(epoch)

In [ ]:
# Create components

from models.sentiment_model import ModelTrainer
model = loaded_model
trainer = ModelTrainer(model=model)
evaluator = ModelEvaluator(model, trainer)

# Run evaluation
eval_results = evaluator.evaluate_model(analyzer.test_texts, analyzer.test_labels)
evaluator.visualize_performance(eval_results, training_history)

In [ ]:
from transformers import DistilBertTokenizer
from main import SentimentAnalyzer# Create tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Initialize analyzer with both model and tokenizer
analyzer = SentimentAnalyzer()
analyzer.model = loaded_model
analyzer.tokenizer = tokenizer

In [ ]:
def test_sentiment_edge_cases(analyzer):
   """
   Test sentiment analyzer with various edge cases organized by sentiment and linguistic pattern.
   
   Args:
       analyzer: Initialized SentimentAnalyzer instance
   """
   test_cases = {
       "Positive with Negation": [
           ("I can't deny that this place is amazing. Not a single thing wrong with the food or service!", "Double Negative -> Positive"),
           ("Never had a bad experience here. The food isn't anything less than perfect.", "Negated Negative"),
           ("Not once have I been disappointed by their service. The staff isn't unfriendly at all.", "Complex Negation")
       ],
       
       "Positive with Sarcasm": [
           ("Yeah right, like this restaurant could possibly get ANY better! *chef's kiss* Absolutely perfect!", "Exaggerated Praise"),
           ("Oh sure, just RUIN my diet with your impossibly delicious desserts! How dare you be this good!", "Mock Complaint"),
           ("Just what I needed - another restaurant to be obsessed with! 🙄 Now I'll have to keep coming back!", "Fake Annoyance")
       ],
       
       "Positive with Multipolarity": [
           ("The wait was long but honestly worth every minute. Amazing food and exceptional service!", "Contrast Resolution"),
           ("Small portions and pricey, but the taste makes up for everything. Will definitely return!", "Trade-off Acceptance"),
           ("Noisy atmosphere but incredible food and the best service I've had in years.", "Mixed with Positive Dominance")
       ],
       
       "Negative with Negation": [
           ("The food isn't good at all. Not worth the price and I won't be returning.", "Direct Negative"),
           ("I couldn't find anything special about this place. The service wasn't even close to acceptable.", "Compound Negative"),
           ("Not once did they get our order right. The manager wasn't helpful either.", "Sequential Negative")
       ],
       
       "Negative with Sarcasm": [
           ("Oh fantastic, another overpriced meal with cold food. Just what I was hoping for! 🙄", "Mock Enthusiasm"),
           ("Wow, amazing how they consistently manage to mess up a simple order. Such talent! 😒", "Ironic Praise"),
           ("Five stars for teaching me the true meaning of patience! Best 2-hour wait ever! 🙃", "Exaggerated Compliment")
       ],
       
       "Negative with Multipolarity": [
           ("Great location but terrible food and even worse service. Definitely not returning.", "Location vs Experience"),
           ("Beautiful decor, shame about the rude staff and inedible food.", "Aesthetics vs Function"),
           ("Nice ambiance but overpriced food and disappointing service ruined the experience.", "Environment vs Service")
       ],
       
       "Neutral with Negation": [
           ("The food isn't amazing but isn't terrible either. Just an average experience.", "Balanced Negation"),
           ("Not the best, not the worst. Wouldn't go out of my way to return.", "Double Neutral Negation"),
           ("Can't say it was great, can't say it was bad. Just okay.", "Negated Extremes")
       ],
       
       "Neutral with Sarcasm": [
           ("Well, that was... an experience. I guess that's one way to run a restaurant! 🤔", "Ambiguous Evaluation"),
           ("'Interesting' take on Italian food. Very... unique interpretation! 😏", "Noncommittal Sarcasm"),
           ("Such a... memorable experience. Definitely different from what I expected! 🫤", "Understated Sarcasm")
       ],
       
       "Neutral with Multipolarity": [
           ("Good food but slow service. Bad parking but nice location. Evens out I guess.", "Balanced Trade-offs"),
           ("Some dishes were great, others terrible. Service varied. Hard to form an opinion.", "Mixed Experience"),
           ("Excellent appetizers, mediocre mains, poor desserts. Averages out to okay.", "Quality Variation")
       ]
   }
   
   for category, cases in test_cases.items():
       print(f"\n=== Testing {category} ===\n")
       for text, case_type in cases:
           prediction = analyzer.predict(text)
           print(f"Case Type: {case_type}")
           print(f"Text: {text}")
           print(f"Prediction: {prediction}")
           print()

# Usage
test_sentiment_edge_cases(analyzer)